## Colton Lee, Eli Herigon, Jake Schildknecht

# Play Type Counts

#### This notebook is used to determine the play type by the top 10 and bottom 10 teams for each play in a season and count how many times the play happened. Also, getting the percentages associated with the count to account for teams having more plays.

In [1]:
import pandas as pd

## Play type classification function

In [2]:
def play_type(playOutcome):
    play = str(playOutcome).lower()
    if "pass" in play:
        return "Pass"
    elif "rush" in play or "run" in play or "ran" in play:
        return "Run"
    elif "punt" in play:
        return "Punt"
    elif "field goal" in play:
        return "Field Goal"
    elif "intercepted" in play or "fumble" in play or "turnover" in play:
        return "Turnover"
    elif "touchdown" in play:
        return "Touchdown"
    elif "timeout" in play:
        return "Timeout"
    else:
        return "Other"

## Play count for top 10 and bottom 10 teams function

In [3]:
def play_count(scoresURL, playsURL):
    # Import data
    scores = pd.read_csv(scoresURL)
    plays = pd.read_csv(playsURL)
    plays["ShortName"] = plays["TeamWithPossession"].apply(lambda x: str(x).split()[-1])

    # Get top 10 and bottom 10 teams
    top10 = scores.head(10)["Team"].tolist()
    bottom10 = scores.tail(10)["Team"].tolist()

    # Get key columns and apply function
    filteredPlays = plays[["Season", "ShortName", "PlayOutcome", "PlayDescription"]].copy()
    filteredPlays["PlayType"] = filteredPlays["PlayOutcome"].apply(play_type)

    # Filter plays by top 10 and bottom 10
    topPlays = filteredPlays[filteredPlays["ShortName"].isin(top10)]
    bottomPlays = filteredPlays[filteredPlays["ShortName"].isin(bottom10)]

    # Top 10 and bottom 10 play counts
    topCount = topPlays.groupby(["ShortName", "PlayType"]).size().unstack(fill_value=0)
    bottomCount = bottomPlays.groupby(["ShortName", "PlayType"]).size().unstack(fill_value=0)

    return topCount, bottomCount

# Play count percentages for Top 10 and Bottom 10 teams function

In [4]:
def play_count_percentage(scoresURL, playsURL):
    # Import data
    scores = pd.read_csv(scoresURL)
    plays = pd.read_csv(playsURL)
    plays["ShortName"] = plays["TeamWithPossession"].apply(lambda x: str(x).split()[-1])

    # Get top 10 and bottom 10 teams
    top10 = scores.head(10)["Team"].tolist()
    bottom10 = scores.tail(10)["Team"].tolist()

    # Get key columns and apply function
    filteredPlays = plays[["Season", "ShortName", "PlayOutcome", "PlayDescription"]].copy()
    filteredPlays["PlayType"] = filteredPlays["PlayOutcome"].apply(play_type)

    # Drop others table because its irrelevant to the percentage tables
    filteredPlays = filteredPlays[filteredPlays["PlayType"] != "Other"].copy()

    # Filter plays by top 10 and bottom 10
    topPlays = filteredPlays[filteredPlays["ShortName"].isin(top10)]
    bottomPlays = filteredPlays[filteredPlays["ShortName"].isin(bottom10)]

    # Top 10 and bottom 10 play counts
    topCount = topPlays.groupby(["ShortName", "PlayType"]).size().unstack(fill_value=0)
    bottomCount = bottomPlays.groupby(["ShortName", "PlayType"]).size().unstack(fill_value=0)

    # Convert counts to percentages of all plays (excluding the "other" column
    topPct_num = topCount.div(topCount.sum(axis=1), axis=0) * 100
    bottomPct_num = bottomCount.div(bottomCount.sum(axis=1), axis=0) * 100

    # Round and add '%' sign
    topPct = topPct_num.round(2).astype(str) + "%"
    bottomPct = bottomPct_num.round(2).astype(str) + "%"

    return topPct, bottomPct


## 2017 Data

In [5]:
top2017, bottom2017 = play_count("2017_scores.csv", "2017_plays.csv")

In [6]:
top2017

PlayType,Field Goal,Other,Pass,Punt,Run,Timeout,Touchdown,Turnover
ShortName,,,,,,,,
Chiefs,53,312,653,87,497,72,52,10
Eagles,49,328,773,96,648,85,70,22
Jaguars,46,290,686,121,722,70,57,12
Patriots,54,348,813,81,606,79,78,13
Rams,54,297,654,90,542,68,56,16
Ravens,46,243,634,109,584,89,46,10
Saints,49,280,706,96,568,88,60,16
Seahawks,40,320,621,103,527,61,51,8
Steelers,45,289,712,89,534,78,57,14


In [7]:
bottom2017

PlayType,Field Goal,Other,Pass,Punt,Run,Timeout,Touchdown,Turnover
ShortName,,,,,,,,
Bears,26,292,562,108,520,55,35,18
Browns,25,325,667,104,467,78,39,15
Cardinals,49,321,700,113,535,75,43,12
Colts,40,293,585,111,517,72,35,10
Dolphins,26,310,665,107,443,60,44,16
Giants,38,278,716,111,486,56,32,17
Jets,38,290,600,124,518,74,43,10
Raiders,26,291,643,85,430,66,46,20
Texans,29,333,593,112,511,49,45,16


In [8]:
#2017 percentage tables
topPct2017, bottomPct2017 = play_count_percentage("2017_scores.csv", "2017_plays.csv")

In [9]:
topPct2017

PlayType,Field Goal,Pass,Punt,Run,Timeout,Touchdown,Turnover
ShortName,,,,,,,
Chiefs,3.72%,45.86%,6.11%,34.9%,5.06%,3.65%,0.7%
Eagles,2.81%,44.35%,5.51%,37.18%,4.88%,4.02%,1.26%
Jaguars,2.68%,40.02%,7.06%,42.12%,4.08%,3.33%,0.7%
Patriots,3.13%,47.16%,4.7%,35.15%,4.58%,4.52%,0.75%
Rams,3.65%,44.19%,6.08%,36.62%,4.59%,3.78%,1.08%
Ravens,3.03%,41.77%,7.18%,38.47%,5.86%,3.03%,0.66%
Saints,3.1%,44.6%,6.06%,35.88%,5.56%,3.79%,1.01%
Seahawks,2.83%,44.01%,7.3%,37.35%,4.32%,3.61%,0.57%
Steelers,2.94%,46.57%,5.82%,34.92%,5.1%,3.73%,0.92%


In [10]:
bottomPct2017

PlayType,Field Goal,Pass,Punt,Run,Timeout,Touchdown,Turnover
ShortName,,,,,,,
Bears,1.96%,42.45%,8.16%,39.27%,4.15%,2.64%,1.36%
Browns,1.79%,47.81%,7.46%,33.48%,5.59%,2.8%,1.08%
Cardinals,3.21%,45.84%,7.4%,35.04%,4.91%,2.82%,0.79%
Colts,2.92%,42.7%,8.1%,37.74%,5.26%,2.55%,0.73%
Dolphins,1.91%,48.86%,7.86%,32.55%,4.41%,3.23%,1.18%
Giants,2.61%,49.18%,7.62%,33.38%,3.85%,2.2%,1.17%
Jets,2.7%,42.64%,8.81%,36.82%,5.26%,3.06%,0.71%
Raiders,1.98%,48.86%,6.46%,32.67%,5.02%,3.5%,1.52%
Texans,2.14%,43.76%,8.27%,37.71%,3.62%,3.32%,1.18%


## 2018 Data

In [11]:
top2018, bottom2018 = play_count("2018_scores.csv", "2018_plays.csv")

In [12]:
top2018

PlayType,Field Goal,Other,Pass,Punt,Run,Timeout,Touchdown,Turnover
ShortName,,,,,,,,
Bears,45,328,682,87,589,112,59,17
Chargers,41,324,644,90,533,86,66,8
Chiefs,38,363,730,65,490,91,84,10
Colts,40,338,798,82,547,72,69,20
Patriots,46,307,787,96,673,76,68,11
Rams,56,339,725,82,634,98,67,13
Ravens,58,317,684,94,708,95,52,15
Saints,43,292,641,60,611,81,75,13
Steelers,30,303,734,83,430,74,67,12


In [13]:
bottom2018

PlayType,Field Goal,Other,Pass,Punt,Run,Timeout,Touchdown,Turnover
ShortName,,,,,,,,
49ers,40,329,612,85,508,72,46,17
Bills,33,334,579,103,554,64,46,11
Buccaneers,35,340,711,71,471,64,64,13
Cardinals,25,287,558,121,433,52,35,17
Dolphins,32,305,570,104,439,67,45,16
Falcons,30,299,694,84,420,56,54,18
Jets,43,288,587,104,489,67,39,17
Lions,38,287,690,93,479,50,46,13
Raiders,40,312,632,92,468,76,40,21


In [14]:
#2018 percentage tables
topPct2018, bottomPct2018 = play_count_percentage("2018_scores.csv", "2018_plays.csv")

In [15]:
topPct2018

PlayType,Field Goal,Pass,Punt,Run,Timeout,Touchdown,Turnover
ShortName,,,,,,,
Bears,2.83%,42.87%,5.47%,37.02%,7.04%,3.71%,1.07%
Chargers,2.79%,43.87%,6.13%,36.31%,5.86%,4.5%,0.54%
Chiefs,2.52%,48.41%,4.31%,32.49%,6.03%,5.57%,0.66%
Colts,2.46%,49.02%,5.04%,33.6%,4.42%,4.24%,1.23%
Patriots,2.62%,44.79%,5.46%,38.3%,4.33%,3.87%,0.63%
Rams,3.34%,43.28%,4.9%,37.85%,5.85%,4.0%,0.78%
Ravens,3.4%,40.09%,5.51%,41.5%,5.57%,3.05%,0.88%
Saints,2.82%,42.06%,3.94%,40.09%,5.31%,4.92%,0.85%
Steelers,2.1%,51.33%,5.8%,30.07%,5.17%,4.69%,0.84%


In [16]:
bottomPct2018

PlayType,Field Goal,Pass,Punt,Run,Timeout,Touchdown,Turnover
ShortName,,,,,,,
49ers,2.9%,44.35%,6.16%,36.81%,5.22%,3.33%,1.23%
Bills,2.37%,41.65%,7.41%,39.86%,4.6%,3.31%,0.79%
Buccaneers,2.45%,49.76%,4.97%,32.96%,4.48%,4.48%,0.91%
Cardinals,2.01%,44.96%,9.75%,34.89%,4.19%,2.82%,1.37%
Dolphins,2.51%,44.78%,8.17%,34.49%,5.26%,3.53%,1.26%
Falcons,2.21%,51.18%,6.19%,30.97%,4.13%,3.98%,1.33%
Jets,3.19%,43.61%,7.73%,36.33%,4.98%,2.9%,1.26%
Lions,2.7%,48.97%,6.6%,34.0%,3.55%,3.26%,0.92%
Raiders,2.92%,46.17%,6.72%,34.19%,5.55%,2.92%,1.53%


## 2019 Data

In [17]:
top2019, bottom2019 = play_count("2019_scores.csv", "2019_plays.csv")

In [18]:
top2019

PlayType,Field Goal,Other,Pass,Punt,Run,Timeout,Touchdown,Turnover
ShortName,,,,,,,,
49ers,51,337,592,74,693,109,74,15
Bills,40,292,643,99,583,83,48,18
Chiefs,45,360,774,74,503,69,76,18
Cowboys,48,279,717,75,514,58,54,15
Packers,26,341,698,106,529,89,62,16
Patriots,44,258,735,98,576,68,59,14
Ravens,44,300,566,56,723,55,68,10
Saints,50,295,694,80,513,67,59,10
Titans,24,368,589,116,618,58,67,17


In [19]:
bottom2019

PlayType,Field Goal,Other,Pass,Punt,Run,Timeout,Touchdown,Turnover
ShortName,,,,,,,,
Bengals,39,313,739,91,444,57,42,23
Cardinals,41,319,657,89,452,78,49,11
Dolphins,40,343,684,87,444,64,42,17
Giants,22,317,699,85,437,46,59,21
Jaguars,38,296,685,101,462,41,33,23
Jets,37,304,610,107,475,60,36,13
Lions,36,339,637,104,481,51,44,10
Panthers,40,317,715,94,445,69,49,24
Raiders,33,256,629,90,539,76,47,15


In [20]:
#2019 percentage tables
topPct2019, bottomPct2019 = play_count_percentage("2019_scores.csv", "2019_plays.csv")

In [21]:
topPct2019

PlayType,Field Goal,Pass,Punt,Run,Timeout,Touchdown,Turnover
ShortName,,,,,,,
49ers,3.17%,36.82%,4.6%,43.1%,6.78%,4.6%,0.93%
Bills,2.64%,42.47%,6.54%,38.51%,5.48%,3.17%,1.19%
Chiefs,2.89%,49.65%,4.75%,32.26%,4.43%,4.87%,1.15%
Cowboys,3.24%,48.41%,5.06%,34.71%,3.92%,3.65%,1.01%
Packers,1.7%,45.74%,6.95%,34.67%,5.83%,4.06%,1.05%
Patriots,2.76%,46.11%,6.15%,36.14%,4.27%,3.7%,0.88%
Ravens,2.89%,37.19%,3.68%,47.5%,3.61%,4.47%,0.66%
Saints,3.39%,47.11%,5.43%,34.83%,4.55%,4.01%,0.68%
Titans,1.61%,39.56%,7.79%,41.5%,3.9%,4.5%,1.14%


In [22]:
bottomPct2019

PlayType,Field Goal,Pass,Punt,Run,Timeout,Touchdown,Turnover
ShortName,,,,,,,
Bengals,2.72%,51.5%,6.34%,30.94%,3.97%,2.93%,1.6%
Cardinals,2.98%,47.71%,6.46%,32.82%,5.66%,3.56%,0.8%
Dolphins,2.9%,49.64%,6.31%,32.22%,4.64%,3.05%,1.23%
Giants,1.61%,51.06%,6.21%,31.92%,3.36%,4.31%,1.53%
Jaguars,2.75%,49.53%,7.3%,33.41%,2.96%,2.39%,1.66%
Jets,2.77%,45.59%,8.0%,35.5%,4.48%,2.69%,0.97%
Lions,2.64%,46.74%,7.63%,35.29%,3.74%,3.23%,0.73%
Panthers,2.79%,49.79%,6.55%,30.99%,4.81%,3.41%,1.67%
Raiders,2.31%,44.02%,6.3%,37.72%,5.32%,3.29%,1.05%


## 2020 Data

In [23]:
top2020, bottom2020 = play_count("2020_scores.csv", "2020_plays.csv")

In [24]:
top2020

PlayType,Field Goal,Other,Pass,Punt,Run,Timeout,Touchdown,Turnover
ShortName,,,,,,,,
Bills,43,288,656,51,443,66,62,12
Buccaneers,40,289,693,67,468,79,74,6
Chiefs,35,273,693,56,449,71,60,11
Colts,39,235,559,51,462,57,50,9
Packers,20,253,548,50,474,65,74,9
Rams,34,230,602,78,508,74,47,13
Ravens,35,242,425,58,589,67,57,10
Saints,31,252,551,64,515,71,63,17
Seahawks,26,279,532,69,419,70,59,9


In [25]:
bottom2020

PlayType,Field Goal,Other,Pass,Punt,Run,Timeout,Touchdown,Turnover
ShortName,,,,,,,,
Bengals,34,259,547,71,391,60,35,11
Broncos,35,247,508,65,423,59,37,14
Cowboys,41,233,595,49,407,63,43,16
Eagles,19,300,553,73,378,53,40,16
Giants,32,233,491,63,380,35,26,16
Jaguars,25,236,571,65,324,35,34,18
Jets,28,231,466,82,392,50,28,10
Lions,28,269,543,59,341,52,47,10
Panthers,36,238,514,42,380,63,38,15


In [26]:
#2020 percentage tables
topPct2020, bottomPct2020 = play_count_percentage("2020_scores.csv", "2020_plays.csv")

In [27]:
topPct2020

PlayType,Field Goal,Pass,Punt,Run,Timeout,Touchdown,Turnover
ShortName,,,,,,,
Bills,3.23%,49.21%,3.83%,33.23%,4.95%,4.65%,0.9%
Buccaneers,2.8%,48.56%,4.7%,32.8%,5.54%,5.19%,0.42%
Chiefs,2.55%,50.4%,4.07%,32.65%,5.16%,4.36%,0.8%
Colts,3.18%,45.56%,4.16%,37.65%,4.65%,4.07%,0.73%
Packers,1.61%,44.19%,4.03%,38.23%,5.24%,5.97%,0.73%
Rams,2.51%,44.4%,5.75%,37.46%,5.46%,3.47%,0.96%
Ravens,2.82%,34.25%,4.67%,47.46%,5.4%,4.59%,0.81%
Saints,2.36%,42.0%,4.88%,39.25%,5.41%,4.8%,1.3%
Seahawks,2.2%,44.93%,5.83%,35.39%,5.91%,4.98%,0.76%


In [28]:
bottomPct2020

PlayType,Field Goal,Pass,Punt,Run,Timeout,Touchdown,Turnover
ShortName,,,,,,,
Bengals,2.96%,47.61%,6.18%,34.03%,5.22%,3.05%,0.96%
Broncos,3.07%,44.52%,5.7%,37.07%,5.17%,3.24%,1.23%
Cowboys,3.38%,49.01%,4.04%,33.53%,5.19%,3.54%,1.32%
Eagles,1.68%,48.85%,6.45%,33.39%,4.68%,3.53%,1.41%
Giants,3.07%,47.08%,6.04%,36.43%,3.36%,2.49%,1.53%
Jaguars,2.33%,53.26%,6.06%,30.22%,3.26%,3.17%,1.68%
Jets,2.65%,44.13%,7.77%,37.12%,4.73%,2.65%,0.95%
Lions,2.59%,50.28%,5.46%,31.57%,4.81%,4.35%,0.93%
Panthers,3.31%,47.24%,3.86%,34.93%,5.79%,3.49%,1.38%


## 2021 Data

In [29]:
top2021, bottom2021 = play_count("2021_scores.csv", "2021_plays.csv")

In [30]:
top2021

PlayType,Field Goal,Other,Pass,Punt,Run,Timeout,Touchdown,Turnover
ShortName,,,,,,,,
49ers,41,325,612,89,661,108,60,23
Bengals,56,367,726,93,589,91,68,12
Bills,38,298,770,65,551,77,77,13
Buccaneers,39,302,873,87,453,72,75,17
Chiefs,41,349,842,61,551,89,78,17
Colts,45,277,585,70,565,80,55,17
Cowboys,43,339,788,88,572,64,66,13
Patriots,49,287,615,62,564,70,64,10
Rams,51,319,781,79,596,94,69,11


In [31]:
bottom2021

PlayType,Field Goal,Other,Pass,Punt,Run,Timeout,Touchdown,Turnover
ShortName,,,,,,,,
Bears,32,303,584,78,515,64,41,21
Falcons,33,293,592,75,447,64,38,19
Giants,34,291,640,91,465,51,32,19
Jaguars,33,289,705,83,427,66,38,16
Jets,33,303,640,83,460,73,43,15
Lions,34,314,638,76,479,76,43,17
Panthers,39,291,653,86,496,68,39,22
Raiders,51,321,725,79,482,77,46,16
Steelers,48,304,785,100,534,73,44,17


In [32]:
#2021 percentage tables
topPct2021, bottomPct2021 = play_count_percentage("2021_scores.csv", "2021_plays.csv")

In [33]:
topPct2021

PlayType,Field Goal,Pass,Punt,Run,Timeout,Touchdown,Turnover
ShortName,,,,,,,
49ers,2.57%,38.39%,5.58%,41.47%,6.78%,3.76%,1.44%
Bengals,3.43%,44.4%,5.69%,36.02%,5.57%,4.16%,0.73%
Bills,2.39%,48.4%,4.09%,34.63%,4.84%,4.84%,0.82%
Buccaneers,2.41%,54.02%,5.38%,28.03%,4.46%,4.64%,1.05%
Chiefs,2.44%,50.15%,3.63%,32.82%,5.3%,4.65%,1.01%
Colts,3.18%,41.28%,4.94%,39.87%,5.65%,3.88%,1.2%
Cowboys,2.63%,48.23%,5.39%,35.01%,3.92%,4.04%,0.8%
Patriots,3.42%,42.89%,4.32%,39.33%,4.88%,4.46%,0.7%
Rams,3.03%,46.46%,4.7%,35.46%,5.59%,4.1%,0.65%


In [34]:
bottomPct2021

PlayType,Field Goal,Pass,Punt,Run,Timeout,Touchdown,Turnover
ShortName,,,,,,,
Bears,2.4%,43.75%,5.84%,38.58%,4.79%,3.07%,1.57%
Falcons,2.6%,46.69%,5.91%,35.25%,5.05%,3.0%,1.5%
Giants,2.55%,48.05%,6.83%,34.91%,3.83%,2.4%,1.43%
Jaguars,2.41%,51.54%,6.07%,31.21%,4.82%,2.78%,1.17%
Jets,2.45%,47.51%,6.16%,34.15%,5.42%,3.19%,1.11%
Lions,2.49%,46.81%,5.58%,35.14%,5.58%,3.15%,1.25%
Panthers,2.78%,46.54%,6.13%,35.35%,4.85%,2.78%,1.57%
Raiders,3.46%,49.12%,5.35%,32.66%,5.22%,3.12%,1.08%
Steelers,3.0%,49.03%,6.25%,33.35%,4.56%,2.75%,1.06%


## 2022 Data

In [35]:
top2022, bottom2022 = play_count("2022_scores.csv", "2022_plays.csv")

In [36]:
top2022

PlayType,Field Goal,Other,Pass,Punt,Run,Timeout,Touchdown,Turnover
ShortName,,,,,,,,
49ers,44,305,640,80,643,96,62,23
Bengals,44,334,770,80,509,101,63,11
Bills,36,301,688,60,515,80,67,18
Chiefs,42,306,787,79,539,95,75,11
Cowboys,40,306,655,89,653,72,68,11
Dolphins,44,298,662,74,456,82,54,11
Eagles,30,348,670,78,703,104,82,19
Lions,37,281,674,59,529,74,61,15
Raiders,46,301,653,77,529,65,52,9


In [37]:
bottom2022

PlayType,Field Goal,Other,Pass,Punt,Run,Timeout,Touchdown,Turnover
ShortName,,,,,,,,
Bears,32,290,419,83,622,55,48,20
Broncos,43,312,649,104,492,75,36,11
Buccaneers,43,264,860,102,465,73,38,17
Cardinals,38,316,735,80,479,87,43,14
Colts,44,299,642,84,486,58,40,22
Rams,32,289,565,83,472,65,42,12
Seahawks,45,334,665,85,503,76,52,19
Texans,36,277,602,101,466,43,43,14
Titans,29,293,514,107,542,70,40,14


In [38]:
#2022 percentage tables
topPct2022, bottomPct2022 = play_count_percentage("2022_scores.csv", "2022_plays.csv")

In [39]:
topPct2022

PlayType,Field Goal,Pass,Punt,Run,Timeout,Touchdown,Turnover
ShortName,,,,,,,
49ers,2.77%,40.3%,5.04%,40.49%,6.05%,3.9%,1.45%
Bengals,2.79%,48.8%,5.07%,32.26%,6.4%,3.99%,0.7%
Bills,2.46%,46.99%,4.1%,35.18%,5.46%,4.58%,1.23%
Chiefs,2.58%,48.34%,4.85%,33.11%,5.84%,4.61%,0.68%
Cowboys,2.52%,41.25%,5.6%,41.12%,4.53%,4.28%,0.69%
Dolphins,3.18%,47.87%,5.35%,32.97%,5.93%,3.9%,0.8%
Eagles,1.78%,39.74%,4.63%,41.7%,6.17%,4.86%,1.13%
Lions,2.55%,46.51%,4.07%,36.51%,5.11%,4.21%,1.04%
Raiders,3.21%,45.63%,5.38%,36.97%,4.54%,3.63%,0.63%


In [40]:
bottomPct2022

PlayType,Field Goal,Pass,Punt,Run,Timeout,Touchdown,Turnover
ShortName,,,,,,,
Bears,2.5%,32.76%,6.49%,48.63%,4.3%,3.75%,1.56%
Broncos,3.05%,46.03%,7.38%,34.89%,5.32%,2.55%,0.78%
Buccaneers,2.69%,53.82%,6.38%,29.1%,4.57%,2.38%,1.06%
Cardinals,2.57%,49.8%,5.42%,32.45%,5.89%,2.91%,0.95%
Colts,3.2%,46.66%,6.1%,35.32%,4.22%,2.91%,1.6%
Rams,2.52%,44.45%,6.53%,37.14%,5.11%,3.3%,0.94%
Seahawks,3.11%,46.02%,5.88%,34.81%,5.26%,3.6%,1.31%
Texans,2.76%,46.13%,7.74%,35.71%,3.3%,3.3%,1.07%
Titans,2.2%,39.06%,8.13%,41.19%,5.32%,3.04%,1.06%


## 2023 Data

In [41]:
top2023, bottom2023 = play_count("2023_scores.csv", "2023_plays.csv")

In [42]:
top2023

PlayType,Field Goal,Other,Pass,Punt,Run,Timeout,Touchdown,Turnover
ShortName,,,,,,,,
49ers,38,331,633,74,616,90,75,17
Bills,37,299,691,66,630,79,68,13
Chiefs,50,319,832,84,576,82,63,15
Cowboys,42,318,722,55,552,64,65,11
Dolphins,31,316,635,62,520,69,71,22
Jaguars,40,299,663,66,517,65,51,27
Lions,29,325,764,85,612,88,75,13
Packers,40,305,673,69,574,86,61,15
Ravens,44,315,583,84,639,78,70,22


In [43]:
bottom2023

PlayType,Field Goal,Other,Pass,Punt,Run,Timeout,Touchdown,Turnover
ShortName,,,,,,,,
Cardinals,39,309,629,77,534,70,43,13
Commanders,28,332,689,84,413,67,52,17
Falcons,39,277,586,89,566,68,34,17
Giants,36,319,576,106,509,57,36,18
Jets,48,304,685,112,463,61,28,28
Panthers,33,303,653,95,494,46,29,24
Patriots,26,265,584,114,460,55,36,16
Rams,50,308,662,81,534,89,54,8
Titans,37,332,536,88,525,65,40,10


In [44]:
#2023 percentage tables
topPct2023, bottomPct2023 = play_count_percentage("2023_scores.csv", "2023_plays.csv")

In [45]:
topPct2023

PlayType,Field Goal,Pass,Punt,Run,Timeout,Touchdown,Turnover
ShortName,,,,,,,
49ers,2.46%,41.02%,4.8%,39.92%,5.83%,4.86%,1.1%
Bills,2.34%,43.62%,4.17%,39.77%,4.99%,4.29%,0.82%
Chiefs,2.94%,48.88%,4.94%,33.84%,4.82%,3.7%,0.88%
Cowboys,2.78%,47.78%,3.64%,36.53%,4.24%,4.3%,0.73%
Dolphins,2.2%,45.04%,4.4%,36.88%,4.89%,5.04%,1.56%
Jaguars,2.8%,46.4%,4.62%,36.18%,4.55%,3.57%,1.89%
Lions,1.74%,45.86%,5.1%,36.73%,5.28%,4.5%,0.78%
Packers,2.64%,44.33%,4.55%,37.81%,5.67%,4.02%,0.99%
Ravens,2.89%,38.36%,5.53%,42.04%,5.13%,4.61%,1.45%


In [46]:
bottomPct2023

PlayType,Field Goal,Pass,Punt,Run,Timeout,Touchdown,Turnover
ShortName,,,,,,,
Cardinals,2.78%,44.77%,5.48%,38.01%,4.98%,3.06%,0.93%
Commanders,2.07%,51.04%,6.22%,30.59%,4.96%,3.85%,1.26%
Falcons,2.79%,41.89%,6.36%,40.46%,4.86%,2.43%,1.22%
Giants,2.69%,43.05%,7.92%,38.04%,4.26%,2.69%,1.35%
Jets,3.37%,48.07%,7.86%,32.49%,4.28%,1.96%,1.96%
Panthers,2.4%,47.53%,6.91%,35.95%,3.35%,2.11%,1.75%
Patriots,2.01%,45.24%,8.83%,35.63%,4.26%,2.79%,1.24%
Rams,3.38%,44.79%,5.48%,36.13%,6.02%,3.65%,0.54%
Titans,2.84%,41.2%,6.76%,40.35%,5.0%,3.07%,0.77%


## 2024 Data

In [47]:
top2024, bottom2024 = play_count("2024_scores.csv", "2024_plays.csv")

In [48]:
top2024

PlayType,Field Goal,Other,Pass,Punt,Run,Timeout,Touchdown,Turnover
ShortName,,,,,,,,
Bills,42,342,634,73,646,89,77,8
Broncos,40,284,630,90,548,80,55,13
Buccaneers,38,320,622,65,556,61,66,12
Chargers,50,286,597,94,522,73,44,8
Commanders,58,360,706,68,670,87,67,16
Eagles,49,333,583,76,763,127,71,18
Lions,37,302,632,60,616,83,79,14
Packers,48,291,543,64,614,74,59,18
Ravens,34,312,532,75,680,72,75,15


In [49]:
bottom2024

PlayType,Field Goal,Other,Pass,Punt,Run,Timeout,Touchdown,Turnover
ShortName,,,,,,,,
Browns,31,334,728,100,439,62,38,20
Cowboys,56,311,704,68,509,66,38,18
Falcons,48,289,608,62,553,66,44,12
Giants,31,301,648,93,473,42,39,18
Jaguars,36,263,588,77,501,58,40,18
Panthers,30,316,586,89,454,61,47,15
Patriots,39,307,597,88,502,57,36,20
Raiders,46,299,710,82,426,58,38,17
Saints,37,293,610,91,508,72,44,7


In [50]:
#2024 percentage tables
topPct2024, bottomPct2024 = play_count_percentage("2024_scores.csv", "2024_plays.csv")

In [51]:
topPct2024

PlayType,Field Goal,Pass,Punt,Run,Timeout,Touchdown,Turnover
ShortName,,,,,,,
Bills,2.68%,40.41%,4.65%,41.17%,5.67%,4.91%,0.51%
Broncos,2.75%,43.27%,6.18%,37.64%,5.49%,3.78%,0.89%
Buccaneers,2.68%,43.8%,4.58%,39.15%,4.3%,4.65%,0.85%
Chargers,3.6%,43.01%,6.77%,37.61%,5.26%,3.17%,0.58%
Commanders,3.47%,42.22%,4.07%,40.07%,5.2%,4.01%,0.96%
Eagles,2.9%,34.56%,4.51%,45.23%,7.53%,4.21%,1.07%
Lions,2.43%,41.55%,3.94%,40.5%,5.46%,5.19%,0.92%
Packers,3.38%,38.24%,4.51%,43.24%,5.21%,4.15%,1.27%
Ravens,2.29%,35.87%,5.06%,45.85%,4.86%,5.06%,1.01%


In [52]:
bottomPct2024

PlayType,Field Goal,Pass,Punt,Run,Timeout,Touchdown,Turnover
ShortName,,,,,,,
Browns,2.19%,51.34%,7.05%,30.96%,4.37%,2.68%,1.41%
Cowboys,3.84%,48.25%,4.66%,34.89%,4.52%,2.6%,1.23%
Falcons,3.45%,43.65%,4.45%,39.7%,4.74%,3.16%,0.86%
Giants,2.31%,48.21%,6.92%,35.19%,3.12%,2.9%,1.34%
Jaguars,2.73%,44.61%,5.84%,38.01%,4.4%,3.03%,1.37%
Panthers,2.34%,45.71%,6.94%,35.41%,4.76%,3.67%,1.17%
Patriots,2.91%,44.59%,6.57%,37.49%,4.26%,2.69%,1.49%
Raiders,3.34%,51.56%,5.95%,30.94%,4.21%,2.76%,1.23%
Saints,2.7%,44.56%,6.65%,37.11%,5.26%,3.21%,0.51%
